In [1]:
# !mkdir 01_Fastqc
# !mkdir 02_Mapping
# !mkdir 02_Mapping/stats
# !mkdir 03_fastq_files
# !mkdir 04_Kraken
import os
import glob
fastq = glob.glob('00_data/*.fastq.gz')
fastq.sort()
with open("run.sh", "w") as rsh:
    for fasti in range(0, len(fastq), 2):
        fast_1 = fastq[fasti]
        fast_2 = fastq[fasti +1]
        sample = fast_1[fast_1.index("/"):fast_1.index("_L001_R1_001.fastq.gz")]
        rsh.write("echo I am processing sample: {}\n".format(fast_1))
#         rsh.write("fastqc -t 65 -o 01_Fastqc/ {}\n".format(fast_1))
#         rsh.write("fastqc -t 65 -o 01_Fastqc/ {}\n".format(fast_1))
#         rsh.write("bwa mem -t 72 BWA_Index/GRCh38_latest_genomic.fna {} {} | samtools view --threads 72 -b | samtools sort --threads 72 -o 02_Mapping{}.bam\n".format(fast_1, fast_2, sample))
#         rsh.write("samtools index 02_Mapping{}.bam 02_Mapping{}.bam.bai\n".format(sample, sample))
#         rsh.write("samtools flagstat --threads 72 02_Mapping/{}.bam > 02_Mapping/stats/{}.txt\n".format(sample, sample))
#         rsh.write("samtools fastq -1 03_fastq_files{}_1.fq -2 03_fastq_files{}_2.fq -0 /dev/null -s /dev/null -n -f 4 02_Mapping{}.bam\n ".format(sample, sample, sample))
        rsh.write("kraken2 --use-mpa-style --threads 60 --db /ramdisk/$DBNAME --report 04_Kraken{}_MetaPhlan_Big.tax --paired  03_fastq_files{}_1.fq 03_fastq_files{}_2.fq > 04_Kraken{}Metaphlan_Big.txt\n".format(sample, sample, sample, sample))
        rsh.write("kraken2 --use-names --threads 60 --db /ramdisk/$DBNAME --report 04_Kraken{}_KrakenOut_Big.tax --paired  03_fastq_files{}_1.fq 03_fastq_files{}_2.fq > 04_Kraken{}Kraken_Big.txt\n".format(sample, sample, sample, sample))

In [10]:
import os
import glob
fastq = glob.glob('00_data/*.fastq.gz')
fastq.sort()
with open("run3.sh", "w") as rsh:
    for fasti in range(0, len(fastq), 2):
        fast_1 = fastq[fasti]
        fast_2 = fastq[fasti +1]
        sample = fast_1[fast_1.index("/"):fast_1.index("_L001_R1_001.fastq.gz")]
        rsh.write("kraken2 --use-mpa-style --memory-mapping --threads 60 --db /ramdisk/k2_eupathdb48_20201113 --report 04_Kraken{}_MetaPhlan_Big.tax --paired  03_fastq_files{}_1.fq 03_fastq_files{}_2.fq > 04_Kraken{}Metaphlan_Big.txt\n".format(sample, sample, sample, sample))
        rsh.write("kraken2 --use-names --memory-mapping --threads 60 --db /ramdisk/k2_eupathdb48_20201113 --report 04_Kraken{}_KrakenOut_Big.tax --paired  03_fastq_files{}_1.fq 03_fastq_files{}_2.fq > 04_Kraken{}Kraken_Big.txt\n".format(sample, sample, sample, sample))
        rsh.write("kraken2 --use-mpa-style --memory-mapping --threads 60 --db /ramdisk/k2_eupathdb48_20201113 --report 04_Kraken{}_MetaPhlan_Big_Human.tax --paired  {} {} > 04_Kraken{}Metaphlan_Big_Human.txt\n".format(sample, fast_1, fast_2, sample))
        rsh.write("kraken2 --use-names --memory-mapping --threads 60 --db /ramdisk/k2_eupathdb48_20201113 --report 04_Kraken{}_KrakenOut_Big_Human.tax --paired  {} {} > 04_Kraken{}Kraken_Big_Human.txt\n".format(sample, fast_1, fast_2, sample))


In [11]:
!bash run3.sh

Loading database information... done.
3154 sequences (0.58 Mbp) processed in 0.850s (222.6 Kseq/m, 40.95 Mbp/m).
  3 sequences classified (0.10%)
  3151 sequences unclassified (99.90%)
Loading database information... done.
3154 sequences (0.58 Mbp) processed in 0.852s (222.0 Kseq/m, 40.84 Mbp/m).
  3 sequences classified (0.10%)
  3151 sequences unclassified (99.90%)
Loading database information... done.
703350 sequences (193.86 Mbp) processed in 5.745s (7345.1 Kseq/m, 2024.45 Mbp/m).
  166 sequences classified (0.02%)
  703184 sequences unclassified (99.98%)
Loading database information... done.
703350 sequences (193.86 Mbp) processed in 5.480s (7700.4 Kseq/m, 2122.36 Mbp/m).
  166 sequences classified (0.02%)
  703184 sequences unclassified (99.98%)
Loading database information... done.
1177 sequences (0.25 Mbp) processed in 0.469s (150.6 Kseq/m, 31.77 Mbp/m).
  1 sequences classified (0.08%)
  1176 sequences unclassified (99.92%)
Loading database information... done.
1177 sequences 

In [35]:
import os
import glob
krakens = glob.glob('/home/genomics/Documents/DR_Amany/new/04_Kraken/Kraken2krona/*.txt')
for kraken in krakens:
    with open(kraken) as rtxt:
        lines = rtxt.readlines()
    with open("{}_without_human.krona".format(kraken[:-4]), "w") as wtxt:
        for line in lines:
            if "(taxid 0)" not in line:
                if "Homo" not in line:
                    tmp = line.split("\t")[1:3]
                    read_id = tmp[0]
                    tax_id = tmp[1]
                    tax_id = tax_id[tax_id.index("taxid")+6:tax_id.index(")")]
                    tmp_w = "{}\t{}\n".format(read_id, tax_id)
                    wtxt.write(tmp_w)


In [36]:
for kraken in krakens:
    print("ktImportTaxonomy {}_without_human.krona -o {}_without_human.html".format(kraken[:-4], kraken[:-4]))

ktImportTaxonomy /home/genomics/Documents/DR_Amany/new/04_Kraken/Kraken2krona/Sample_2_without_human.krona -o /home/genomics/Documents/DR_Amany/new/04_Kraken/Kraken2krona/Sample_2_without_human.html
ktImportTaxonomy /home/genomics/Documents/DR_Amany/new/04_Kraken/Kraken2krona/Sample_3_without_human.krona -o /home/genomics/Documents/DR_Amany/new/04_Kraken/Kraken2krona/Sample_3_without_human.html
ktImportTaxonomy /home/genomics/Documents/DR_Amany/new/04_Kraken/Kraken2krona/Sample_4_without_human.krona -o /home/genomics/Documents/DR_Amany/new/04_Kraken/Kraken2krona/Sample_4_without_human.html
ktImportTaxonomy /home/genomics/Documents/DR_Amany/new/04_Kraken/Kraken2krona/Sample_1_without_human.krona -o /home/genomics/Documents/DR_Amany/new/04_Kraken/Kraken2krona/Sample_1_without_human.html
